這份 Notebook 示範 OpenAI Function Calling 基本用法


In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)

  if response.status_code == 200 :
    return obj["choices"][0]["message"] # 這裡有小改，變成回傳上一層的 message 物件
  else :
    return obj["error"]

## 基本的完整流程

參考自官方案例 https://openai.com/blog/function-calling-and-other-api-updates

### 使用 Google Search 工具

In [4]:
# 這是非官方的 google 爬蟲
!pip install googlesearch-python

# 若要用官方 JSON API https://developers.google.com/custom-search/v1/overview?hl=zh-tw (有 API key 需付費但有免費額度)

  Preparing metadata (setup.py) ... done
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=d8845e03eaa3eebed2c45d7b77eae775ae5cd68a3e0373e7110c7ac40e7c4d9d
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python


In [5]:
from googlesearch import search

## Step 1

定義工具函式

In [6]:
def google_search(keyword):
  content = ""
  for item in search(keyword, advanced=True, num_results=5, lang='zh-tw'):
    content += f"Title: {item.title}\n Description: {item.description}\n\n"
  return content

接著發出 Prompt 請求，附上你有的 function 規格

其中 parameters 部分是個 JSON schema 格式 https://json-schema.org/

In [7]:
messages = [{"role": "user", "content": "今天台北天氣如何?"}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "google_search",
            "description": "搜尋最新的資訊",
            "parameters": { # 這是一個 json schema 格式
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜尋關鍵字",
                    }
                },
                "required": ["keyword"],
            },
        },
    }
]

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_oNeL6gnA69W1mgdgyoXQdfJB',
                 'type': 'function',
                 'function': {'name': 'google_search',
                              'arguments': '{"keyword":"台北天氣"}'}}]}


## Step 2

解析回傳拿到工具參數，實際呼叫 get_current_weather 方法，得到工具結果

In [8]:
metadata = json.loads(response["tool_calls"][0]["function"]["arguments"])
pp(metadata)

{'keyword': '台北天氣'}


In [9]:
# 要把 AI 的回覆加到對話歷史紀錄
messages.append(response)

# 函數名稱與函數物件的對應
available_functions = {
  "google_search": google_search,
}

# response 若有 tool_calls，表示 GPT 要我呼叫函數
if response.get("tool_calls"):
  for tool_call in response.get("tool_calls"):
    # GPT 要我執行的函數名稱
    function_name = tool_call["function"]["name"]

    # 這是函數物件
    fuction_to_call = available_functions[function_name]

    # 擷取出函式的參數
    function_args = json.loads(tool_call["function"]["arguments"])

    # 實際呼叫函數
    function_response = fuction_to_call(**function_args)

    # 將函數的回傳結果，也塞回對話紀錄，角色是 tool
    messages.append(
        {
            "tool_call_id": tool_call["id"],
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    pp(function_response)

('Title: 縣市預報- 臺北市\n'
 ' Description: 今日白天 陰短暫雨 17 - 1863 - 64降雨機率30%稍有寒意; 今晚明晨 多雲時陰 15 - 1759 - '
 '63降雨機率20%寒冷至稍有寒意; 明日白天 陰時多雲短暫陣雨 15 - 1959 - 66降雨\xa0...\n'
 '\n'
 'Title: 臺北市, 台北市, 臺灣每小時天氣\n'
 ' Description: RealFeel Shade™63°. 風東北11英里/小时. 空氣品質一般. 最大紫外線指數0 低. 陣風14英里/小时. '
 '濕度69%. 露點56° F. 雲層80%. 能見度7英里. 雲冪2800英尺\xa0...\n'
 '\n'
 'Title: 臺北市, 台灣- 氣象預報| 地圖\n'
 ' Description: 星期, 天氣狀況, 降雨機率, 溫度. 檢視氣象預報詳細資訊 星期六, 陣雨. 降雨機率: 51%. 華氏高溫: 65°; '
 '攝氏高溫: 19°; 華氏低溫: 54°; 攝氏低溫: 13°.\n'
 '\n'
 'Title: 東榮, 臺北市10 天天氣預報\n'
 ' Description: 早陣雨。 低溫16°C。 15 到30 公里/小時的東南東風。 降雨機率為50%。\n'
 '\n'
 'Title: 臺北市, 台北市, 臺灣三日天氣預報\n'
 ' Description: 臺北市, 台北市, 臺灣Weather Forecast, with current conditions, wind, '
 'air quality, and what to expect for the next 3 days.\n'
 '\n'
 'Title: 東榮, 臺北市每小時天氣預報\n'
 ' Description: 體感溫度13°. 風向東19 km/h. 濕度69%. 紫外線指數0 (最大值11). 雲量75%. 雨量0 公厘. '
 '05:00. 15°. 3%. 多雲時晴. 體感溫度14°. 風向東19 km/h. 濕度70%.\n'
 '\n'
 'Title: 臺北市信義區天氣預報,臺灣七日氣象溫度,降雨機率\n'
 ' Description: '
 '今天（10日）清晨天氣仍偏冷，白天起強烈

## Step 3

 再次呼叫 OpenAI API，此時 messages 裡面有三個訊息: user, assistant 和 function 結果:

In [10]:
messages

[{'role': 'user', 'content': '今天台北天氣如何?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_oNeL6gnA69W1mgdgyoXQdfJB',
    'type': 'function',
    'function': {'name': 'google_search',
     'arguments': '{"keyword":"台北天氣"}'}}]},
 {'tool_call_id': 'call_oNeL6gnA69W1mgdgyoXQdfJB',
  'role': 'tool',
  'name': 'google_search',
  'content': 'Title: 縣市預報- 臺北市\n Description: 今日白天 陰短暫雨 17 - 1863 - 64降雨機率30%稍有寒意; 今晚明晨 多雲時陰 15 - 1759 - 63降雨機率20%寒冷至稍有寒意; 明日白天 陰時多雲短暫陣雨 15 - 1959 - 66降雨\xa0...\n\nTitle: 臺北市, 台北市, 臺灣每小時天氣\n Description: RealFeel Shade™63°. 風東北11英里/小时. 空氣品質一般. 最大紫外線指數0 低. 陣風14英里/小时. 濕度69%. 露點56° F. 雲層80%. 能見度7英里. 雲冪2800英尺\xa0...\n\nTitle: 臺北市, 台灣- 氣象預報| 地圖\n Description: 星期, 天氣狀況, 降雨機率, 溫度. 檢視氣象預報詳細資訊 星期六, 陣雨. 降雨機率: 51%. 華氏高溫: 65°; 攝氏高溫: 19°; 華氏低溫: 54°; 攝氏低溫: 13°.\n\nTitle: 東榮, 臺北市10 天天氣預報\n Description: 早陣雨。 低溫16°C。 15 到30 公里/小時的東南東風。 降雨機率為50%。\n\nTitle: 臺北市, 台北市, 臺灣三日天氣預報\n Description: 臺北市, 台北市, 臺灣Weather Forecast, with current conditions, wind, air quality,

In [11]:
response = get_completion(messages)
pp(response)

{'role': 'assistant',
 'content': '今天台北的天氣為陰短暫雨，白天氣溫介於17至18度，降雨機率為30%，稍有寒意。晚上到明天清晨則為多雲時陰，氣溫介於15至17度，降雨機率為20%，寒冷至稍有寒意。明天白天則為陰時多雲短暫陣雨，氣溫介於15至19度，降雨機率為59至66%。記得攜帶雨具和增添衣物以應對天氣變化哦。'}


## 換一個問題試看看

如果問題沒有需要呼叫 function 呢?

In [12]:
messages.append( {"role": "user", "content": "寫一首關於天氣的詩"} )

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': '在這個晴朗的日子裡，\n'
            '陽光溫暖，微風吹拂著樹葉。\n'
            '天空湛藍，白雲飄過，\n'
            '大自然的美麗，讓人心情愉快。\n'
            '\n'
            '雨滴輕輕地落下，\n'
            '清澈的水珠在窗戶上跳舞。\n'
            '雷聲隨之響起，\n'
            '大地在雨中得到滋潤。\n'
            '\n'
            '無論是晴天或雨天，\n'
            '天氣總是讓人感到驚喜。\n'
            '讓我們珍惜每一個天氣的瞬間，\n'
            '感受大自然的奇妙與美麗。'}


回傳的 response 裡面沒有 tool_calls，表示 GPT 不需要我們執行工具